## Die gesammelten Daten bereinigen

Im nächsten Schritt müssen die gesammelten Daten in ein Format gebracht werden, das für neuronale Netzwerke geeignet ist. Genauer wollen wir die JSON-Datei in eine CSV-Datei umwandeln.

Die Tabelle, die wir bekommen wollen, sollte ca. folgendermassen aussehen:

| id | firstname | firstname | yearOfBirth | gender | ... | 32277 | 32219 | 32256 | 32259 | ... |
|-|-|-|-|-|-|-|-|-|-|-|
| "58916" | "Jorgo" | "Ananiadis" | 1969 | 0.0 | ... | 0.0 | 0.25 | 0.25 | 0.75 | ... |
| "55096" | "Jacqueline" | "Badran" | 1961 | 1.0 | ... | 0.0 | 1.0 | 1.0 | 1.0 | ... |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |

Die Spalten, die Zahlen als Überschrift haben, enthalten die Antworten der Kandidierenden.

Hier kommt eine neue Bibliothek ins Spiel:
[`pandas`](https://de.wikipedia.org/wiki/Pandas_(Software)) ist eine Programmbibliothek für Python zur Verarbeitung, Analyse und Darstellung von Daten.

In [ ]:
import json
import pandas as pd


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename_json = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"
filename_csv_all = "nationalrat_all.csv" if mode == 1 else "ständerat_all.csv"
filename_csv_train = "nationalrat_train.csv" if mode == 1 else "ständerat_train.csv"
filename_csv_test = "nationalrat_test.csv" if mode == 1 else "ständerat_test.csv"

Zuerst Laden wir wieder die Rohdaten und schauen uns wieder ein\*e Kandidat*in an.

In [ ]:
candidates = json.load(open(filename_json))
candidates[0]

Die Antworten auf die Fragen sind aktuell im Format `{'questionId': '32225', 'value': 25}`.
Die `questionId`s sind aus mir unbekannten Gründen scheinbar zufällig angeordnet, lasst uns die also einmal sortieren.

In [ ]:
for candidate in candidates:
    candidate["answers"].sort(key=lambda answer: answer["questionId"])
candidates[0]

Neuronale Netzwerke "mögen" aus mathematischen Gründen normalisierte Zahlen. Heisst: Zahlen zwischen 0 und 1. 
- Für die Antworten ist die Umwandlung trivial, da die `value` Felder bereits alle zwischen 0 und 100 sind.
- Das Netzwerk soll auch das Geschlecht als Input erhalten, also wandeln wir `"m"` und `"f"` in `0` und `1` um. Einige Kandidierenden haben als Geschlecht `"x"` angegeben, das wandeln wir in `0.5` um.

In [ ]:
for candidate in candidates:
    for answer in candidate["answers"]:
        answer["value"] = answer["value"] / 100
    if candidate["gender"] == "m":
        candidate["gender"] = 0
        continue
    if candidate["gender"] == "f":
        candidate["gender"] = 1
        continue
    candidate["gender"] = 0.5
candidates[0]

Aktuell sind unsere Daten noch hierarchisch verschachtelt. Um die Tabelle erstellen zu können, müssen die Einträge von `'answers'` "herausgezogen" werden.

In [ ]:
for candidate in candidates:
    for answer in candidate["answers"]:
        candidate[answer["questionId"]] = answer["value"]
    del candidate["answers"]
candidates[0]

Vorhin beim Analysieren der Daten ist aufgefallen, dass von einigen Parteien nur sehr wenige Personen kandidieren. Lasst uns also alle Kandidierenden aussortieren, die weniger als 9 Kameraden haben.

In [ ]:
from collections import defaultdict


party_count = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

considered_parties = set([party_name for party_name, count in party_count.items() if count >= 10])

candidates = [candidate for candidate in candidates if candidate["partyAbbreviation"] in considered_parties]

Als letztes speichern wir die Daten als CSV-Dateien ab.

Dabei unterteilen wir die Kandidierenden per Zufallsprinzip in eine Trainingsmenge und in eine Testmenge, das wird später beim Trainieren und Testen des neuronalen Netwerks wichtig.

In [ ]:
candidates_table = pd.DataFrame(candidates)
candidates_table.to_csv(filename_csv_all)


from sklearn.model_selection import train_test_split


train, test = train_test_split(candidates_table, test_size=0.25, stratify=[candidate["partyAbbreviation"] for candidate in candidates])
train.to_csv(filename_csv_train)
test.to_csv(filename_csv_test)